# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

# Load Secrets

In [1]:
%load_ext dotenv
%dotenv ../05_src/.secrets_grassriots

## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [ ]:
# Import document cleaning utilities from utils module
import sys
sys.path.append('../05_src')

from utils.document_cleaning import (
    clean_document_text,
    evaluate_cleaning,
    print_evaluation_report
)

# Import PyPDFLoader from langchain_community
from langchain_community.document_loaders import PyPDFLoader

file_path = "https://latitude-blog.ghost.io/blog/10-examples-of-tone-adjusted-prompts-for-llms/"
loader = PyPDFLoader(file_path)
docs = loader.load()

# Combine all page content into a single string
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"


In [20]:
# Note: All cleaning and evaluation functions are now imported from utils.document_cleaning module
# Functions are available: clean_document_text, evaluate_cleaning, print_evaluation_report

# I wanted to explore the impact of the cleaning process on the document length - and how it affects the LLM's ability to summarize the document.
# Specifically, how we can optimize the amount of input tokens for the LLM, while still maintaining the quality of the summary.

# Store original text before cleaning for comparison
original_document_text = document_text

# Clean the document text before processing with LLM
document_text = clean_document_text(document_text)

print(f"Original document length: {len(original_document_text)} characters")
print(f"Cleaned document length: {len(document_text)} characters")
print()

# Evaluate the cleaning effectiveness
cleaning_metrics = evaluate_cleaning(original_document_text, document_text)
print_evaluation_report(cleaning_metrics)


Original document length: 51452 characters
Cleaned document length: 50434 characters

DOCUMENT CLEANING EVALUATION REPORT

📊 TEXT STATISTICS
--------------------------------------------------------------------------------
Character Count:
  Original: 51,452
  Cleaned:  50,434
  Reduction: 1,018 (1.98%)

Word Count:
  Original: 8,670
  Cleaned:  8,427

Sentence Count:
  Original: 578
  Cleaned:  578

Average Words per Sentence:
  Original: 15.00
  Cleaned:  14.58

🔤 WHITESPACE METRICS
--------------------------------------------------------------------------------
Spaces:
  Original: 7,759
  Cleaned:  8,426
  Reduced:  -667

Multiple Consecutive Spaces:
  Original: 5
  Cleaned:  0
  Reduced:  5

Tab Characters:
  Original: 0
  Cleaned:  0
  Removed:  0

Single Newlines (broken sentences):
  Original: 1,442
  Cleaned:  0
  Fixed:    1,442

Excessive Blank Lines (>2 consecutive):
  Original: 0
  Cleaned:  0
  Removed:  0

✨ TEXT QUALITY INDICATORS
-----------------------------------------

## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [ ]:
# --- Structured Summarization with OpenAI Chat Completions API ---
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional
from openai import OpenAI
import json

# 1) Output Schema - simplified for strict mode
class SummarySchema(BaseModel):
    model_config = ConfigDict(extra='forbid')
    
    Author: str = Field(..., description="Author of the article")
    Title: str = Field(..., description="Title of the article")
    Relevance: str = Field(..., description="One paragraph on relevance to AI professionals")
    Summary: str = Field(..., description="Concise summary, <=1000 tokens")
    Tone: str = Field(..., description="The tone used to produce the summary")
    InputTokens: int = Field(..., description="Token count placeholder")
    OutputTokens: int = Field(..., description="Token count placeholder")

# 2) Model + Tone
MODEL_NAME = "gpt-4o-mini"
TONE = "Skeptical and Cynical"

# 3) Prompts (separated) + dynamic context injection
instructions = (
    "You are an information extraction and summarization assistant. "
    "Return output STRICTLY matching the provided JSON schema. "
    "Do not add fields. Do not include extra commentary. "
    "Write the Summary in the specified Tone and keep it under 1000 tokens. "
    "Use only facts from the provided document; avoid speculation or hallucinations."
)

user_template = (
    "Task: Extract metadata and summarize the document in the specified tone.\n"
    "- Tone: {tone}\n"
    "- Fields to fill: Author, Title, Relevance (<=1 paragraph), Summary (<=1000 tokens), Tone.\n"
    "For InputTokens and OutputTokens, always use 0.\n"
    "Document follows between <<< >>>. Use only its content.\n"
    "<<<\n{context}\n>>>"
)
user_content = user_template.format(tone=TONE, context=document_text)

# 4) Call the model with structured output via Chat Completions API
client = OpenAI()

# Get schema and fix it for strict mode
schema = SummarySchema.model_json_schema()

# Manually ensure all properties are in required array
if "required" not in schema or "InputTokens" not in schema.get("required", []):
    schema["required"] = list(schema.get("properties", {}).keys())

# Add additionalProperties: false
schema["additionalProperties"] = False

print("Schema required fields:", schema.get("required"))  # Debug

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": instructions},
        {"role": "user", "content": user_content},
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "SummarySchema",
            "schema": schema,
            "strict": True,
        },
    },
    max_completion_tokens=1200,
)

# 5) Parse JSON text with Pydantic and overwrite token counts
raw_json = response.choices[0].message.content

parsed = SummarySchema.model_validate_json(raw_json)

result = parsed.model_copy(update={
    "Tone": TONE,
    "InputTokens": response.usage.prompt_tokens,
    "OutputTokens": response.usage.completion_tokens,
})

print("Token usage:", response.usage)
print(result.model_dump_json(indent=2))

# 6) Sanity checks
for field in ["Author", "Title", "Relevance", "Summary", "Tone"]:
    assert getattr(result, field) and isinstance(getattr(result, field), str), f"{field} missing/invalid"
assert result.Relevance.count(".") <= 6, "Relevance seems longer than a short paragraph"
assert isinstance(result.InputTokens, int) and isinstance(result.OutputTokens, int), "Token counts missing"
assert result.Tone == TONE, "Tone label not preserved"

# 7) Save JSON
summary_json_path = "assignment_1_summary.json"
with open(summary_json_path, "w") as f:
    f.write(result.model_dump_json(indent=2))


Schema required fields: ['Author', 'Title', 'Relevance', 'Summary', 'Tone', 'InputTokens', 'OutputTokens']
Token usage: CompletionUsage(completion_tokens=346, prompt_tokens=10833, total_tokens=11179, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
{
  "Author": "Peter F. Drucker",
  "Title": "Managing Oneself",
  "Relevance": "This document addresses the crucial need for self-management in today’s knowledge economy, emphasizing individual responsibility in personal career management. AI professionals must recognize the implications of self-awareness and adaptability in navigating their careers, particularly in an ever-evolving technological landscape where traditional corporate structures are losing significance.",
  "Summary": "Peter F. Drucker’s article, \"Managing Oneself,\" explores the significance of self-aw

'assignment_1_summary.json'

# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

Please, do not forget to add your comments.


# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
